In [42]:
import numpy as np
import tensorflow as tf

#입력변수:2개, 출력:3가지 종류


# '기타(o)' 분류기: hf=wo1x1+wo2x2+b
# '포유류(m)' 분류기: hf=wm1x1+wm2x2+b
# '조류(b)' 분류기: hf=wb1x1+wb2x2+b
# w변수가 6개 필요함 => 입력변수:2 * 출력변수:3 =6

#[털(x1), 날개(x2)]
x_data = np.array(
    [[0, 0], 
     [1, 0], 
     [1, 1], 
     [0, 0], 
     [0, 0], 
     [0, 1]])

# [기타, 포유류, 조류]
y_data = np.array([
    [1, 0, 0],  # 기타
    [0, 1, 0],  # 포유류
    [0, 0, 1],  # 조류
    [1, 0, 0],
    [1, 0, 0],
    [0, 0, 1]
])
                #[입력특성개수, 출력레이블개수] => [2,3]
W=tf.Variable(tf.random_normal([2,3]))
b=tf.Variable(tf.zeros([3])) #분류기 개수만큼 존재
x=tf.placeholder(tf.float32)
y=tf.placeholder(tf.float32)

hf=tf.add(tf.matmul(x,W),b) #x:(6,2), w:(2,3), xw:(6,3)
hf=tf.nn.relu(hf)
model=tf.nn.softmax(hf) #[3,2,1.5] => [0.5, 0.3, 0.2]
cost=tf.reduce_mean(-tf.reduce_sum(y*tf.log(model), axis=1))

optimizer=tf.train.GradientDescentOptimizer(0.01)
train=optimizer.minimize(cost)


#sigmoid(입력값을 0~1로 변환), relu함수(음수->0, 양수->양수값 그대로 출력)
#ex) relu(-3)=>0, relu(5)=>5
#ex) sigmoid(-3)=> 0.1, sigmoid(5) => 0.8

# 0.1*0.1*0.1*...*0.00001 => 0에 가까워
# h(x)->relu->h(x)->sig->0~1->...->0이 되기도 함-> 신호손실
# h(x)->relu->h(x)->relu->h(x)->...->sigmoid()->0~1사이로 출력

In [43]:
init=tf.global_variables_initializer()
sess=tf.Session()
sess.run(init)

In [44]:
for step in range(100):
    sess.run(train, feed_dict={x:x_data, y:y_data})
    if(step+1)%10==0:
        print(step+1, sess.run(cost, feed_dict={x:x_data, y:y_data}))

10 1.045753
20 1.0356992
30 1.026197
40 1.0171872
50 1.0086634
60 1.0005485
70 0.9928198
80 0.9855463
90 0.97858644
100 0.97194886


In [45]:
#결과 확인
#[[0 1 0] [0 0 1]] => argmax => [1 2]
prediction=tf.argmax(model,1)
target=tf.argmax(y,1)
print("예측값:", sess.run(prediction, feed_dict={x:x_data}))
print("실제값:", sess.run(target, feed_dict={y:y_data}))

예측값: [0 2 2 0 0 2]
실제값: [0 1 2 0 0 2]


In [46]:
is_correct=tf.equal(prediction, target)
accuracy=tf.reduce_mean(tf.cast(is_correct, tf.float32))
print("정확도:%.2f" % sess.run(accuracy*100, feed_dict={x:x_data, y:y_data}))

정확도:83.33


In [47]:
from tensorflow.examples.tutorials.mnist import input_data
import random
import matplotlib.pyplot as plt

In [65]:
mnist=input_data.read_data_sets("MNIST_data/", one_hot=True)
nb_classes=10 #0~9
x=tf.placeholder(tf.float32, shape=[None, 784]) #784 픽셀 = 28*28이미지
y=tf.placeholder(tf.float32, shape=[None, nb_classes])
w=tf.Variable(tf.random_normal([784,nb_classes]))
b=tf.Variable(tf.random_normal([nb_classes]))
hf= tf.nn.softmax(tf.matmul(x,w)+b ) #[0.01, 0.05, 0.1, 0.4 ... 0.01]
cost= tf.reduce_mean(-tf.reduce_sum(y*tf.log(hf),1))
train= tf.train.GradientDescentOptimizer(0.1).minimize(cost)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [66]:
iscorrect=tf.equal(tf.argmax(hf,1), tf.argmax(y,1))
#실제값 3, 예측 2=> True
accuracy=tf.reduce_mean(tf.cast(iscorrect, tf.float32))

In [68]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(15): #15번 반복
        costavr=0
        totalbatch=int(mnist.train.num_examples/100) #6만/100 => 600
        for i in range(totalbatch):
            batchx, batchy = mnist.train.next_batch(100)#한번에 100개의 이미지를 읽어들임
            cv, _ = sess.run([cost, train], feed_dict={x:batchx, y:batchy})
            costavr += cv/totalbatch #cost 평균
        print("에폭:", "%4d"%(epoch+1), "cost=", "{:.9f}".format(costavr))
    print("학습종료")

에폭:    1 cost= 2.861383367
에폭:    2 cost= 1.152871760
에폭:    3 cost= 0.905346750
에폭:    4 cost= 0.783815769
에폭:    5 cost= 0.709042422
에폭:    6 cost= 0.656738854
에폭:    7 cost= 0.617194898
에폭:    8 cost= 0.585002257
에폭:    9 cost= 0.560035474
에폭:   10 cost= 0.538697980
에폭:   11 cost= 0.520537528
에폭:   12 cost= 0.504862914
에폭:   13 cost= 0.490215856
에폭:   14 cost= 0.478052206
에폭:   15 cost= 0.467702240
학습종료
